# Итоговый проект. Улицы. Объекты культурного наследия.

In [1]:
# TODO собрать установку всех необходимых модулей в одном месте
%pip install geopandas 
%pip install mapclassify
%pip install osmnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
%pip install jupyter_contrib_nbextensions
%pip install jupyterlab
%pip install keplergl
!jupyter labextension install @jupyter-widgets/jupyterlab-manager keplergl-jupyter
!jupyter contrib nbextension install --user
!jupyter nbextension install --py --sys-prefix keplergl
!jupyter nbextension enable --py --sys-prefix keplergl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
-\|/-\|/-\|/-\|/--\|/-\|/Building jupyterlab assets (production, minimized)
-\|/-\|/-\|/-\-\|/-\|/--\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\|/-\

In [3]:
!pip install git+https://github.com/giswqs/leafmap.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/giswqs/leafmap.git to /tmp/pip-req-build-btr58z75
  Running command git clone -q https://github.com/giswqs/leafmap.git /tmp/pip-req-build-btr58z75


In [4]:
# TODO собрать импорты всех модулей в одном месте
#import json
#import requests  
import geopandas as gpd
import mapclassify 
import osmnx as ox

from keplergl import KeplerGl as KGL
from google.colab import output

In [5]:
from IPython.display import Javascript
import leafmap.kepler as leafmap

In [6]:
TILES = "CartoDB positron"  # Название подложки для карт

# TODO указать любой район Санкт-Петербург из OSM https://wiki.openstreetmap.org/wiki/RU:%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B
TERRITORY_NAME = "Красногвардейский район"  # название территории для которой будут строиться слои

# TODO указать ссылку на файл из вашего github репозитория, которая начинается с https://raw.githubusercontent.com/
KGIOP_FILE_URL = "https://raw.githubusercontent.com/vitalybar/PythonUrbanITMO2022/main/geojson_layers/kgiop_objects.geojson"  # ссылка на слой с объектами культурного наследия
STREETS_FILE_URL = "https://raw.githubusercontent.com/vitalybar/PythonUrbanITMO2022/main/geojson_layers/streets.geojson"   # ссылка на слой с улицами

## Территория

### Загрузка территории из OSM (Extract)

In [7]:
# TODO загрузить geodataframe с геометрией для территории TERRITORY_NAME
osm_id = "R1114895"  # https://www.openstreetmap.org/relation/1114895
district = ox.geocode_to_gdf([osm_id], by_osmid=True)
district

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((30.39428 59.92902, 30.39591 59.92574...",60.040946,59.915431,30.566029,30.394281,298252125,relation,1114895,59.978272,30.486575,"Красногвардейский район, Saint Petersburg, Nor...",boundary,administrative,0.400226


In [8]:
distr_center = [district["lat"].values[0], district["lon"].values[0]]

## Улицы

### Загрузка файла с улицами (Extract)

In [9]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
gdf_streets = gpd.read_file(STREETS_FILE_URL, mask=district)

gdf_streets.rename(columns={"name": "Название"}, inplace=True)
gdf_streets.head()

,Название,geometry
0,Шепетовская улица,"LINESTRING (3385685.754 8387961.891, 3385692.2..."
1,Екатерининский проспект,"LINESTRING (3388641.888 8396122.519, 3388624.9..."
2,проспект Энергетиков,"LINESTRING (3388144.278 8393976.488, 3388156.8..."
3,Пискарёвский проспект,"LINESTRING (3390821.701 8403547.048, 3390778.0..."
4,проспект Маршала Блюхера,"LINESTRING (3388626.448 8394297.512, 3388806.4..."


### Обработка данных с улицами (Transform)

In [10]:
gdf_streets["Название"].is_unique

False

In [11]:
# TODO сгруппировать и объединить геометрии с одинаковыми названиями
gdf_streets = gdf_streets.dissolve(by="Название")
gdf_streets.head()

,geometry
Название,
1-й Охтинский путепровод,"LINESTRING (3385032.231 8389412.855, 3385026.9..."
1-я Жерновская улица,"LINESTRING (3393354.687 8391674.447, 3393424.7..."
1-я Поперечная улица,"LINESTRING (3400575.014 8394720.248, 3400536.9..."
1-я линия,"LINESTRING (3400408.291 8394100.825, 3400263.2..."
2-й Охтинский путепровод,"LINESTRING (3385001.317 8388918.283, 3384996.1..."


### Сохранение слоя с улицами (Load)

In [12]:
# TODO сохранить слой в географической проекции в формате GeoJSON
gdf_streets.to_crs(4326).to_file('streets.geojson', driver='GeoJSON') 

### Визуализация

In [13]:
gdf_streets.explore(tiles=TILES)

## Объекты культурного наследия

### Загрузка объектов культурного наследия (Extract)

In [14]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
gdf_culture = gpd.read_file(KGIOP_FILE_URL, mask=district)

#gdf_streets.rename(columns={"name": "Название"}, inplace=True)
gdf_culture.head()

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry
0,2733,—,Казармы служителей Охтинского Порохового завод...,20 в. нач.,"2-я Жерновская ул., 58, лит. А; 2-я Жерновская...",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Ансамбль,POINT (3393917.541 8391682.542)
1,2734,Казармы служителей Охтинского Порохового завод...,Казарма,20 в. нач.,"2-я Жерновская ул., 58, лит. А",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3393942.031 8391697.888)
2,2735,Казармы служителей Охтинского Порохового завод...,Казарма,20 в. нач.,"2-я Жерновская ул., 64, лит. А",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3393893.051 8391664.084)
3,2736,—,Доходный дом П.И. Иванова,1906-1907,"Большая Пороховская ул., 16; Среднеохтинский п...",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3385859.012 8389439.579)
4,2737,—,Особняк П.И. Иванова,1901,"Большая Пороховская ул., 18",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3385918.345 8389472.261)


### Обработка объектов культурного наследия (Transform)

In [15]:
# TODO добавить два столбца lon и lat, в которых будут долгота и широта 
gdf_culture["lon"] = gdf_culture.to_crs(3857).geometry.centroid.to_crs(4326).x
gdf_culture["lat"] = gdf_culture.to_crs(3857).geometry.centroid.to_crs(4326).y
gdf_culture.head()

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry,lon,lat
0,2733,—,Казармы служителей Охтинского Порохового завод...,20 в. нач.,"2-я Жерновская ул., 58, лит. А; 2-я Жерновская...",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Ансамбль,POINT (3393917.541 8391682.542),30.488080,59.963799
1,2734,Казармы служителей Охтинского Порохового завод...,Казарма,20 в. нач.,"2-я Жерновская ул., 58, лит. А",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3393942.031 8391697.888),30.488300,59.963868
2,2735,Казармы служителей Охтинского Порохового завод...,Казарма,20 в. нач.,"2-я Жерновская ул., 64, лит. А",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3393893.051 8391664.084),30.487860,59.963716
3,2736,—,Доходный дом П.И. Иванова,1906-1907,"Большая Пороховская ул., 16; Среднеохтинский п...",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3385859.012 8389439.579),30.415689,59.953712
4,2737,—,Особняк П.И. Иванова,1901,"Большая Пороховская ул., 18",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3385918.345 8389472.261),30.416222,59.953859


### Сохранение слоя с объектами культурного наследия (Load)

In [16]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
rename_columns = {
    'id': 'Идентификатор',
    'ensemble_name': 'Имя группы',
    'object_name': 'Имя объекта',
    'occurrence_time': 'Время появления',
    'object_location': 'Местоположение объекта',
    'historical_category': 'Иситорическая категория',
    'normative_act': 'Нормативный акт',
    'object_type': 'Тип объекта',
    'lon': 'Долгота',
    'lat': 'Широта'
}
gdf_culture.rename(columns=rename_columns, inplace=True)
gdf_culture.head()

,Идентификатор,Имя группы,Имя объекта,Время появления,Местоположение объекта,Иситорическая категория,Нормативный акт,Тип объекта,geometry,Долгота,Широта
0,2733,—,Казармы служителей Охтинского Порохового завод...,20 в. нач.,"2-я Жерновская ул., 58, лит. А; 2-я Жерновская...",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Ансамбль,POINT (3393917.541 8391682.542),30.488080,59.963799
1,2734,Казармы служителей Охтинского Порохового завод...,Казарма,20 в. нач.,"2-я Жерновская ул., 58, лит. А",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3393942.031 8391697.888),30.488300,59.963868
2,2735,Казармы служителей Охтинского Порохового завод...,Казарма,20 в. нач.,"2-я Жерновская ул., 64, лит. А",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3393893.051 8391664.084),30.487860,59.963716
3,2736,—,Доходный дом П.И. Иванова,1906-1907,"Большая Пороховская ул., 16; Среднеохтинский п...",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3385859.012 8389439.579),30.415689,59.953712
4,2737,—,Особняк П.И. Иванова,1901,"Большая Пороховская ул., 18",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3385918.345 8389472.261),30.416222,59.953859


In [17]:
# TODO сохранить слой в географической проекции в формате GeoJSON
gdf_culture.to_crs(4326).to_file('kgio.geojson', driver='GeoJSON')

###Визуализация

In [18]:
gdf_culture.explore(tiles=TILES)

###KeplerGL

In [20]:
m = leafmap.Map(center = distr_center, zoom=10.5)
streets = 'streets.geojson'
m.add_geojson(streets, layer_name="Улицы")

In [21]:
display(Javascript('''
  google.colab.widgets.installCustomManager('https://ssl.gstatic.com/colaboratory-static/widgets/colab-cdn-widget-manager/6a14374f468a145a/manager.min.js');
'''))

m

<IPython.core.display.Javascript object>

Map(config={'version': 'v1', 'config': {'mapState': {'latitude': 59.9782718, 'longitude': 30.486575093731183, …

In [22]:
m_1 = leafmap.Map(center = distr_center, zoom=10.7)
streets = 'kgio.geojson'
m_1.add_geojson(streets, layer_name="Достопримечательности")

In [23]:
display(Javascript('''
  google.colab.widgets.installCustomManager('https://ssl.gstatic.com/colaboratory-static/widgets/colab-cdn-widget-manager/6a14374f468a145a/manager.min.js');
'''))

m_1

<IPython.core.display.Javascript object>

Map(config={'version': 'v1', 'config': {'mapState': {'latitude': 59.9782718, 'longitude': 30.486575093731183, …